# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [5]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# set the parameters to run the script
directories = ['/media/elea/ssd/NPP/TU/550']
calib_directory = '/media/elea/ssd/NPP/calib'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
parameter_set = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = True

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'no'

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = 'all'

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz

# NB: processing time without PDDN
# 'no_viz': 0.71s
# 'default', save_pdf_figs False: 2.25s
# 'default', save_pdf_figs True: 3.60s
# 'full', save_pdf_figs False: 3.95s
# 'full', save_pdf_figs True: 8.07s
processing_mode = 'default'
save_pdf_figs = False

# define if the wavelenghts should be aligned before processing - and used for the computation
align_wls = False

parameters = batch_processing.get_parameters(directories, calib_directory, wavelengths, parameter_set = parameter_set, 
                                PDDN_mode = PDDN_mode, processing_mode = processing_mode, run_all = run_all, 
                                save_pdf_figs = save_pdf_figs, align_wls = align_wls)

In [20]:
%%time
times = batch_processing.batch_process_master(parameters)
times

processing without PDDN...


  0%|                                                     | 0/1 [00:00<?, ?it/s]

Processing: /media/elea/ssd/NPP/TU/550/2022-09-01_T_HORAO-37-BF_FR_S2_1


100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.81s/it]

processing without PDDN done.
CPU times: user 7.75 s, sys: 547 ms, total: 8.29 s
Wall time: 3.81 s


{'MM_processing': 0.30785202980041504,
 'azimuth_std_processing': 0.01401519775390625,
 'denoising': 0,
 'save_npz': 0.09975314140319824,
 'full_processing': 0.7157979011535645,
 'plotting': 1.9401719570159912,
 'moving': 0.019231319427490234,
 'total': 3.8122122287750244}

In [ ]:
%%time

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
times = batch_processing.batch_visualization(parameters)
times

In [ ]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths
directories = ['/media/elea/ssd/NPP/TU/checked']

PDDN_mode = 'both'
run_all = True
align_wavelengths.align_wavelenghts(directories, PDDN_mode, run_all)

Aligning wavelenght: 600nm


  0%|                                                    | 0/53 [00:00<?, ?it/s]

 
 >> Exported X3D .cod file as: /media/elea/ssd/NPP/TU/checked/2023-11-02_T_HORAO-84-AF_FR_S1_1/raw_data/600nm/600_Intensite_aligned.cod
 
 >> Exported X3D .cod file as: /media/elea/ssd/NPP/TU/checked/2023-11-02_T_HORAO-84-AF_FR_S1_1/raw_data/600nm/600_Intensite_PDDN_aligned.cod


  2%|▊                                           | 1/53 [00:16<13:55, 16.07s/it]

 
 >> Exported X3D .cod file as: /media/elea/ssd/NPP/TU/checked/2023-09-21_T_HORAO-104-AF_FR_S4_1/raw_data/600nm/600_Intensite_aligned.cod


...

LandmarkBasedTransformInitializer (0x5ebf2a7289d0): Less than 2 landmarks available. Rotation is not computed



 
 >> Exported X3D .cod file as: /media/elea/ssd/NPP/TU/checked/2023-06-08_T_HORAO-77-AF_FR_S1_1/raw_data/650nm/650_Intensite_aligned.cod


 87%|█████████████████████████████████████▍     | 47/54 [07:12<00:59,  8.48s/it]

 
 >> Exported X3D .cod file as: /media/elea/ssd/NPP/TU/checked/2023-05-04_T_HORAO-100-BF_FR_S1_1/raw_data/650nm/650_Intensite_aligned.cod


 89%|██████████████████████████████████████▏    | 48/54 [07:28<01:03, 10.64s/it]

In [39]:
# add the calibration matrices (A.cod and W.cod) to the folder for processing
# needed by Christopher
from processingmm.addons import add_calibration
add_calibration.add_calibration(directories)

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 15.59it/s]

No calibration files in 600nm
